### Calculate Datawell MkIII and DWR4 filters

In [ ]:
##using Pkg
##Pkg.add("SpecialFunctions")
##Pkg.add("QuadGK")

#########################################################################
# Using formula contained in Datawell document "Double Integration and Band Pass Filter"
#########################################################################

using Printf
using DSP,Plots
using SpecialFunctions
using QuadGK


# define structure of buoy filter parameters
struct Buoy    
    fₛ::Float64
    fₗₒ::Float64
    fᵤₚ::Float64
    M::Int
    α::Float64
    P₀::Int
    color1::String
    color2::String
    label::String
end
   
# add buoy filter parameters to structure
MkIII = Buoy(3.84, 0.034, 0.61, 511, 8, 178, "palegreen", "darkgreen", "MkIII")
DWR4 = Buoy(2.56, 0.03333, 1.0, 255, 9.3, 123, "darkred", "yellow", "DWR4")

buoy_types = [MkIII, DWR4]

#plot_number = [p1,p2]

# process both DWR4 and MkIII buoys
j = 1
for i in buoy_types
    
    # calc f₀
    f₀ = 1 / i.P₀

    # create empty vector
    global dt = zeros(0)
    global dt₁ = zeros(0)

    # create vector of time values over length of filter
    m = Int(round(i.M/2))
    time = [-m:1:m;]

    print("Processing ",i.label," now.\n")
    # integrate filter expression over range of values
    ## formula (7) in Datawell notes
    for t in time
        
        df(x) = cos(2*π*x*t) / (x^2 + f₀^2);
        append!(dt,-1/(2*π^2)*quadgk(df,i.fₗₒ,i.fᵤₚ)[1])
#=        
        # calculate check result using sine and cosine integrals
        x₀ = 2* π * f₀ * t
        x₁ = 2* π * i.fᵤₚ * t 
        x₂ = 2* π * i.fₗₒ * t  
        
        append!(dt₁,t/π*(sinh(x₀)/2/x₀ * (sinint(x₁ + real(1im*x₀)) + sinint(x₁ - real(1im*x₀))) -
                       real(1im*cosh(x₀))/2/x₀ * (cosint(abs(x₁ + real(1im*x₀))) - cosint(abs(x₁ - 1im*x₀)))) - 
                   t/π*(sinh(x₀)/2/x₀ * (sinint(x₂ + real(1im*x₀)) + sinint(x₂ - real(1im*x₀))) -
                       real(1im*cosh(x₀))/2/x₀ * (cosint(abs(x₂ + 1im*x₀)) - cosint(abs(x₂ - 1im*x₀)))))
=#
    end 

    # generate a Kaiser window over filter range using α value
    global window_function = kaiser(i.M+2, i.α)

    # plot filter with Kaiser window applied to remove Gibbs phenomenon
    if j == 1
        
        p1 = plot(time,dt.*window_function/i.fₛ,title="Datawell filters", 
            c = i.color1, lw = 2, xlim=(-150,150), xticks=(-150:25:150), yticks=(-0.6:0.1:0.2),
            label = i.label, fg_legend = :false, legendfont=font(12),
            size = (950, 650), framestyle = :box)
        
    else
        
        p2 = display(plot!(time,dt.*window_function/i.fₛ, 
            c = i.color1, lw = 1, 
            label = i.label, fg_legend = :false, legendfont=font(12),
            size = (950, 650), framestyle = :box))
        
    end
    
    global j += 1
end

### Calculate DWR-G filter

In [ ]:
#########################################################################
# Using formula contained in Datawell document "Summation filter in the DWR-G"
#########################################################################

using Printf
using DSP,Plots
using SpecialFunctions
using QuadGK

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# define structure of buoy filter parameters
struct Buoy    
    fₛ::Float64
    fₗₒ::Float64
    fᵤₚ::Float64
    M::Int
    α::Float64
end
   
# add buoy filter parameters to structure
DWR_G = Buoy(2, 0.0077, 1, 512, 7.7)

buoy_types = [DWR_G]

# process DWR-G buoy data
   
# create empty vector
ct = zeros(0)

# create vector of time values over length of filter
time = [-DWR_G.M:1/DWR_G.fₛ:DWR_G.M;]

print("Processing DWR_G data now - takes about a minute!\n")
flush(stdout)
# integrate filter expression over range of values
## formula (6) in Datawell notes

######################################
@time begin

    # Create a vector of integrals
    ct = -0.5 * [quadgk(x -> sin(2 * π * x * t) / sin(π * x / DWR_G.fₛ), DWR_G.fₗₒ, DWR_G.fᵤₚ)[1] for t in time]

end
######################################

# generate a Kaiser window over filter range using α value
window_function = kaiser(length(time), DWR_G.α)

# plot filter with Kaiser window applied to remove Gibbs phenomenon
p1 = plot(time, ct, color = :red, lw = :1, label = "Filter", fg_legend = :false, legendfont=font(10))

p1 = plot!(time, ct.*window_function, color=:blue, label="Windowed Filter")
p1 = plot!(time, window_function, color=:lightblue, label="Kaiser Window")
    
plot_p1 = plot(p1, title="Datawell filters",
    xlim=(-250,250), xticks=(-250:50:250),
    ylim=(-1,1), yticks=(-1:0.2:1),
    size = (950, 650), framestyle = :box)
    
display(plot_p1)

### Detailed plots as included in Datawell document

In [ ]:
p1 = plot(time, ct, color = :red, lw = :1,
    label = "Filter", fg_legend = :false, legendfont=font(10),
    xlim=(0,25), xticks=(0:5:25),
    ylim=(-0.7,0), yticks=(-0.7:0.1:0),)
p1 = plot!(time, ct.*window_function, color=:blue, label="Windowed Filter")

p2 = plot(time, ct, color = :red, lw = :1,
    label = "Filter", fg_legend = :false, legendfont=font(10),
    xlim=(0,250), xticks=(0:50:250),
    ylim=(-0.1,0.1), yticks=(-0.1:0.02:0.1),)
p2 = plot!(time, ct.*window_function, color=:blue, label="Windowed Filter")
p2 = plot!(time, window_function, color=:lightblue, label="Kaiser Window")
    
plot_p1 = plot(p1, p2, layout=(1,2), title="DWR-G filters",  
    size = (950, 650), framestyle = :box)
    
display(plot_p1)

### Spectral plot of filter

## Plot Mk4 filter

In [ ]:
#########################################################################
# Using formula contained in Datawell document "Double Integration and Band Pass Filter"
#########################################################################

using Printf
using DSP,Plots
using SpecialFunctions
using QuadGK

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

fₛ = 1.28
fₗₒ= 0.03333
fᵤₚ = 1.0
M = 255
α = 9.3
P₀ = 123
label = "DWR4"

# calc f₀
f₀ = 1 / P₀

# create empty vector
global dt = zeros(0)

# create vector of time values over length of filter
m = Int(round(M/2)) 
time = [-m:1:m;]

print("Processing ",label," now.\n")
# integrate filter expression over range of values - formula (7) in Datawell notes
for t in time

    df(x) = cos(2*π*x*t) / (x^2 + f₀^2);
    append!(dt,-1/(2*π^2)*quadgk(df,fₗₒ,fᵤₚ)[1])

end 

# generate a Kaiser window over filter range using α value
global window_function = kaiser(M+2, α)

dwr4_filter = dt.*window_function/fₛ

# plot filter with Kaiser window applied to remove Gibbs phenomenon
p2 = display(plot(time,dwr4_filter, 
    c=:red, lw=:1, 
    label=label, fg_legend=:false, legendfont=font(12),
    size=(950, 650), framestyle=:box))

In [ ]:
using FFTW

function calc_power_spectrum(signal)
#####################################
    
    N = length(signal)
    dt = 1/sample_frequency
    
    f = fft(signal) |> fftshift
    freqs = fftfreq(N, 1/dt) |> fftshift

    Sxx = 2 * dt ^ 2 / N * (f .* conj(f))

    pos_vals = findall( x -> x >= 0, freqs)
    
    return(freqs[pos_vals],Sxx[pos_vals])
    
end    # calc_power_spectrum()


sample_frequency = fₛ
freqs,Pden1 = calc_power_spectrum(dwr4_filter);

plot(freqs,real.(Pden1),
    fillrange = 0, fillalpha = 0.075, fillcolor = :blue,
    xlim=(0,1), xticks=(0:0.05:1),
    ylim=(0,maximum(real.(Pden1))*1.1), size=(1800,600),
    label="Filter", framestyle = :box, fg_legend = :false, legendfont=font(10))

In [ ]:
# Plot spectrogram over scalogram
nw=128;
spec = DSP.Periodograms.spectrogram(dwr4_filter, nw, 120; fs=2.56, window=hanning);


In [ ]:
1/2.56

In [ ]:
p1 = plot!(time, spec.freq, DSP.Periodograms.power(spec), lw=1, c=cgrad(:Spectral, rev=true), colorbar=false) 

In [ ]:
plot(DSP.Periodograms.power(spec))

In [ ]:
DSP.Periodograms.power(drw4_filter)